In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Dropout, LSTM, GRU,Flatten
from tensorflow.keras.utils import to_categorical

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import FinanceDataReader as fdr

In [6]:
df=fdr.DataReader('005930', '2020', '2025')

In [7]:
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,55500,56000,55000,55200,12993228,-0.010753
2020-01-03,56000,56600,54900,55500,15422255,0.005435
2020-01-06,54900,55600,54600,55500,10278951,0.000000
2020-01-07,55700,56400,55600,55800,10009778,0.005405
2020-01-08,56200,57400,55900,56800,23501171,0.017921
...,...,...,...,...,...,...
2024-07-19,85600,86100,84100,84400,18569122,-0.028769
2024-07-22,84400,84900,82600,83000,18987560,-0.016588
2024-07-23,84200,84700,83400,83900,15766389,0.010843


In [8]:
sc=MinMaxScaler()
df=pd.DataFrame(sc.fit_transform(df),columns=df.columns)
df.head()

,Open,High,Low,Close,Volume,Change
0,0.270440,0.233803,0.269068,0.261856,0.085468,0.315121
1,0.280922,0.245070,0.266949,0.268041,0.114201,0.411142
2,0.257862,0.226291,0.260593,0.268041,0.053361,0.378904
3,0.274633,0.241315,0.281780,0.274227,0.050177,0.410968
4,0.285115,0.260094,0.288136,0.294845,0.209766,0.485209


In [16]:
df['Previous_Close'] = df['Close'].shift(1)
df['Increase'] = (df['Close'] > df['Previous_Close']).astype(int)

In [17]:
df.drop('Previous_Close',axis=1, inplace=True)

In [18]:
df.head()

,Open,High,Low,Close,Volume,Change,Increase
0,0.270440,0.233803,0.269068,0.261856,0.085468,0.315121,0
1,0.280922,0.245070,0.266949,0.268041,0.114201,0.411142,1
2,0.257862,0.226291,0.260593,0.268041,0.053361,0.378904,0
3,0.274633,0.241315,0.281780,0.274227,0.050177,0.410968,1
4,0.285115,0.260094,0.288136,0.294845,0.209766,0.485209,1


In [19]:
window_size=30

x = []
y = []

for i in range(len(df)-window_size-1):
  x.append(df.iloc[i:i+30, :])
  y.append(df.iloc[i+30+1, 6])

In [20]:
x = np.array(x)
y = np.array(y)

In [23]:
len(y)

1096

In [26]:
x.shape

(1096, 30, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      1127 non-null   float64
 1   High      1127 non-null   float64
 2   Low       1127 non-null   float64
 3   Close     1127 non-null   float64
 4   Volume    1127 non-null   float64
 5   Change    1127 non-null   float64
 6   Increase  1127 non-null   int32  
dtypes: float64(6), int32(1)
memory usage: 57.4 KB


In [24]:
x_train=x[:800]
x_test=x[800:]
y_train=y[:800]
y_test=y[800:]

In [43]:
model=Sequential()
model.add(LSTM(256,input_shape=(30,7),return_sequences=True))
model.add(LSTM(256))
model.add(Dense(200,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

c:\Users\M\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 30, 256)        │       270,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 200)            │        51,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 20)             │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 873,439 (3.33 MB)

 Trainable params: 873,439 (3.33 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10


25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.5267 - loss: 0.6932 - val_accuracy: 0.5405 - val_loss: 0.6920
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.4820 - loss: 0.6933 - val_accuracy: 0.5405 - val_loss: 0.6896
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.5255 - loss: 0.6938 - val_accuracy: 0.5405 - val_loss: 0.6909
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.5284 - loss: 0.6926 - val_accuracy: 0.5405 - val_loss: 0.6903
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.4844 - loss: 0.6948 - val_accuracy: 0.5405 - val_loss: 0.6897
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.5426 - loss: 0.6873 - val_accuracy: 0.5203 - val_loss: 0.6928
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.5067 - loss: 0.6929 - val_accuracy: 0.5405 - val_loss: 0.6904
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.5394 - loss: 0.6915 - val_accuracy: 0.5405 - val_loss: 0.

In [38]:
model=Sequential()
model.add(LSTM(256,input_shape=(30,7)))
model.add(Dense(50))
model.add(Dense(20))
model.add(Dense(10))
model.add(Dense(1))
model.summary()

c:\Users\M\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       270,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 50)             │        12,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 20)             │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 284,427 (1.09 MB)

 Trainable params: 284,427 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.4806 - loss: 2.8781 - val_accuracy: 0.4696 - val_loss: 0.7111
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4482 - loss: 0.7330 - val_accuracy: 0.5439 - val_loss: 0.6960
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5204 - loss: 0.7001 - val_accuracy: 0.5135 - val_loss: 0.6963
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5281 - loss: 0.6970 - val_accuracy: 0.5135 - val_loss: 0.6946
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5374 - loss: 0.6954 - val_accuracy: 0.5169 - val_loss: 0.6945
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5266 - loss: 0.6999 - val_accuracy: 0.5135 - val_loss: 0.6945
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5075 - loss: 0.7003 - val_accuracy: 0.5372 - val_loss: 0.6948
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5285 - loss: 0.6920 - val_accuracy: 0.4628 - v